In [3]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from scripts.load_data import load_data

In [16]:
#  correct file path
file_path = "../data/Common Data Warehouse-Orginal.xlsx"

#  Excel file
xls = pd.ExcelFile(file_path)

xls.sheet_names

['Customer', 'Instrument', 'Financial', 'GL', 'Group GL']

In [17]:
# Loading sheets
customer_df = xls.parse("Customer")  # Contains Customer Responsible Unit
financial_df = xls.parse("Financial")  # Contains Amount
gl_df = xls.parse("GL")  # Contains Group GL Name

# Display first few rows
display(customer_df.head())
display(financial_df.head())
display(gl_df.head())


,Customer ID,Local Customer ID,Customer Name,NACE Code L1,NACE Name L1,NACE Code,Bankrupcy Flag,Rating Score,Country,Customer Responsible Unit,Sector Code,Sector Name
0,1000009735298,4339256379,YYT YQRWMZRTS ZT,G,WHOLESALE AND RETAIL TRADE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000009736257,4343649237,FRXSTZDS KZNSZS TYNYLQX T V,G,WHOLESALE AND RETAIL TRADE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000044667284,813518872,SZNDSLX HWLDXNG ZS,L,REAL ESTATE ACTIVITIES,682090.0,0.0,NaN,NO,1.040121e+09,5.0,Non financial corporations
3,1000007802416,992508949,ØSTYRDZLYN GWLFTZNY ZS,R,"ARTS, ENTERTAINMENT AND RECREATION",931100.0,0.0,PPE+,NO,1.040222e+09,5.0,Non financial corporations
4,1000007807925,994176307,JYRRZDW ZS,L,REAL ESTATE ACTIVITIES,682090.0,0.0,NaN,NO,1.040420e+09,5.0,Non financial corporations


,Agmt ID,GL Acct ID,Source,Cust ID,Amount
0,1000162418540,1000000008710,229|ADJ,1000007793706,0.00
1,1000376865946,1000000013691,229,1000009653088,-5198.52
2,1000205191664,1000000008710,229|ADJ,1000044609208,0.00
3,1000329416423,1000000008710,229|ADJ,1000009657652,0.00
4,1000239743539,1000000008710,229|ADJ,1000007699415,0.00


,GL Account ID,GL Account,GL Name,Group GL Acct ID,GL Account Type,GL Account Name
0,1000000008584,100,Loan guarantees,1000000002721,0,Off-balance-sheet items
1,1000000013690,100427,EIR Correction for loan fees paid ni arrears,1000000002728,1,Assets
2,1000000008710,100400,"Loans non-demand, other",1000000002728,1,Assets
3,1000000015433,104417,"Unpaid interest amount on loans, non demand, SLO",1000000103203,1,Assets
4,1000000008590,110,Other guarantees,1000000002721,0,Off-balance-sheet items


In [29]:
# Load Required Libraries and Data ---

import pandas as pd

# file path
file_path = "../data/Common Data Warehouse-Orginal.xlsx"

#  Excel file
xls = pd.ExcelFile(file_path)


customer_df = xls.parse("Customer")  # Contains Customer Responsible Unit
financial_df = xls.parse("Financial")  # Contains Amount
gl_df = xls.parse("GL")  # Contains GL Account Name

#  Merging Customer with Financial Table 

customer_financial_df = financial_df.merge(
    customer_df, left_on="Cust ID", right_on="Customer ID", how="left"
)

#  Merging with GL Table

customer_financial_gl_df = customer_financial_df.merge(
    gl_df, left_on="GL Acct ID", right_on="GL Account ID", how="left"
)

#  Filter for Missing Customer Responsible Units

missing_cust_assets_df = customer_financial_gl_df[
    (customer_financial_gl_df["Customer Responsible Unit"].isnull()) & 
    (customer_financial_gl_df["GL Account Name"] == "Assets")
]

#  Calculating Total Assets for Missing Customer Responsible Units 

total_assets_missing_cust_unit = missing_cust_assets_df["Amount"].sum()

print("🔹 Corrected Total Assets for Missing Customer Responsible Unit:", total_assets_missing_cust_unit)

#  Showing Trends by Customer Sector 


if "Sector Name" in missing_cust_assets_df.columns:
    missing_cust_by_sector = missing_cust_assets_df.groupby("Sector Name").agg(
        Total_Assets=("Amount", "sum"),
        Count_Customers=("Cust ID", "count")
    ).reset_index()

   
    missing_cust_by_sector_str = missing_cust_by_sector.to_string(index=False, header=False)

    #  Results 
    print("\n🔹 Missing Customer Responsible Unit by Sector:")
    print(missing_cust_by_sector_str)

    # Save Results to Excel
    output_file_path = "../data/Missing_Customer_Responsible_Unit_by_Sector.xlsx"
    missing_cust_by_sector.to_excel(output_file_path, index=False, header=False)
    print(f"\n✅ Results saved successfully to: {output_file_path}")

else:
    print("\n🚨 'Sector Name' column is missing in the dataset. Please check the data structure.")


🔹 Corrected Total Assets for Missing Customer Responsible Unit: -3480878046.74

🔹 Missing Customer Responsible Unit by Sector:
Empty DataFrame
Columns: [Sector Name, Total_Assets, Count_Customers]
Index: []

✅ Results saved successfully to: ../data/Missing_Customer_Responsible_Unit_by_Sector.xlsx
